<a href="https://colab.research.google.com/github/annasvenbro/etudesnordiques/blob/main/Test_API_SRU_Sudoc_langue.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Test de l'API SRU du Sudoc pour la présence des fonds en langues étrangères dans les RCR du réseau

##Import des paquets nécessaires

In [1]:
import requests as rq
import xml.etree.ElementTree as et
import pandas as pd
import numpy as np

## Récupérer les données RCR du Sudoc

###Requêter le jeu de données en *open data* d'IdRef pour la liste des RCR via le *webservice* "listrcr" de l'ABES (jeu de données complet *a priori*, avec 2964 entrées).

In [ ]:
liste_rcr=rq.get("https://www.idref.fr/services/listrcr") #On requête l'URL du webservice.
liste_rcr_text=liste_rcr.text #On voit la tête de la réponse.
liste_rcr_text

In [ ]:
lines=liste_rcr_text.split("\n")#Transformation de la réponse en tableau.
header=lines[0].split("\t")
header[0]=header[0].strip("\ufeff") #Pour ne pas avoir de bug dans le dataframe final avec les BOM.
data=[line.split("\t") for line in lines[1:] if line]
df_rcr=pd.DataFrame(data,columns=header)
df_rcr

In [ ]:
df_rcr.columns = df_rcr.columns.str.strip("\ufeff") #Nettoyage du dataframe (BOM, signe égal, guillemets et autres caractères parasites).
df_rcr["RCR"]=df_rcr["RCR"].str.replace("=","")
df_rcr["RCR"]=df_rcr["RCR"].str.replace('"','')
df_rcr= df_rcr.rename(columns={"LONGITUDE\r":"LONGITUDE"})
df_rcr["LONGITUDE"]=df_rcr["LONGITUDE"].str.rstrip("\r")
df_rcr

###Sélection des données pertinentes destinées à alimenter le *dataframe* par RCR à construire pour une langue donnée

Maintenant, il faut ne retenir dans le *dataframe* que 1. le n°RCR de ces bibliothèques, 2.le nom complet, 3. les coordonnées géographiques.

In [ ]:
df_rcr=df_rcr.filter(regex='^RCR$|^LIBELLE$|^LATITUDE$|^LONGITUDE$')#Sinon on a un bug à cause des BOM.
df_rcr

##Établir un *dataframe* avec tous les résultats par RCR pour une langue donnée

###Création de la fonction de requête en fonction du numéro RCR et de la langue

In [ ]:
langue_fr=input("Quelle est la langue dont vous souhaiteriez obtenir une cartographie des fonds dans le Sudoc ? ")#On pose la question de la langue à requêter.

In [ ]:
#À cette variable, on va en associer une autre correspondant au code ISO 639-2 dont se sert l'API du Sudoc pour ses codes de langue.
langues=rq.get("https://www.loc.gov/standards/iso639-2/ISO-639-2_utf-8.txt") #On va donc créer un dataframe avec les codes de langues, à partir de la liste publiée par la Library of Congress.
langues.encoding="utf-8"
langues_text=langues.text.lstrip("\ufeff") #Encore une fois, pour ne pas avoir de bug dans le dataframe final avec les BOM.
langues_text

In [ ]:
lines=langues_text.split("\n") #Création du dataframe des différentes langues.
data_list=[]
for line in lines:
    if line.strip() != "":
        columns = line.split("|")
        data_list.append({
            "Code":columns[0],
            "Bibliographic":columns[1],
            "Terminology":columns[2],
            "French":columns[4]
        })
df_langues=pd.DataFrame(data_list)
df_langues

In [ ]:
langue=df_langues[df_langues["French"].str.lower()==langue_fr.lower()]["Code"].values[0]
langue

Attention !!! API du Sudoc distingue 10 langues dans autres pour lesquelles il faut utiliser la limitation LAN au lieu de LAI dans la requête API. On va donc devoir faire une disjonction entre les langues qu'il faudra requêter avec le code "LAN" et celle avec le code "LAI".

In [ ]:
LAN=["ger","eng","spa","fre","ita","lat","dut","pol","por","rus"] #On définit la liste des codes de langues centrales devant être requêtées avec le code "LAN" dans l'API du Sudoc.

In [ ]:
def get_langue_sudoc(RCR,langue):
  if langue in LAN:
   req=rq.get(f"https://www.sudoc.abes.fr/cbs/sru/?operation=searchRetrieve&version=1.1&recordSchema=unimarc&query=rbc%3D{RCR}%20and%20lan%3D{langue}")
  else:req=rq.get(f"https://www.sudoc.abes.fr/cbs/sru/?operation=searchRetrieve&version=1.1&recordSchema=unimarc&query=rbc%3D{RCR}%20and%20lai%3D{langue}")
  root_sudoc=et.fromstring(req.content)
  for child in root_sudoc.findall("{http://www.loc.gov/zing/srw/}numberOfRecords"):
    return child.text

In [ ]:
get_langue_sudoc(751052116,langue) #On teste la fonction pour le Fonds Général de la BSG.

###Création d'un *dataframe* des résultats pour tous les RCR pour une langue donnée (celle donnée par la réponse à la question "Quelle est la langue dont vous souhaiteriez obtenir une cartographie des fonds dans le Sudoc ? ")

In [ ]:
df_rcr["Notices"]=df_rcr.apply(lambda row: get_langue_sudoc(row["RCR"],langue), axis=1)
df_rcr

La mise à jour du *dataframe* prend un certain temps (33 minutes pour le vietnamien)...

In [ ]:
df_rcr.dtypes #Ce n'est toujours pas propre pour les notices, qui ne sont pas au format numérique.

In [ ]:
df_rcr["Notices"]=pd.to_numeric(df_rcr["Notices"]) #On veut que cette colonne contienne des données numériques.
df_rcr.dtypes# On vérifie.

In [ ]:
df_rcr=df_rcr[df_rcr["Notices"]!=0] #On supprime les lignes des RCR qui n'ont pas de notices dans la langue concernée.
df_rcr

In [ ]:
df_rcr=df_rcr.sort_values(["Notices"],ascending=False)#On retrie le dataframe pour afficher d'abord les RCR ayant le plus grand nombre de documents.
df_rcr

##Représentations cartographiques et diagrammes

###Création du *geodataframe* et carte des RCR qui ont des notices dans la langue concernée

####Installation et importation des paquets nécessaires à la cartographie

In [ ]:
pip install geopandas

In [ ]:
pip install mapclassify

In [ ]:
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from pyproj import CRS
import mapclassify

####Préparation des données du dataframe précédent et création du geodataframe

In [ ]:
df_rcr["LATITUDE"].replace("null",None,inplace=True) #On doit s'occuper des RCR qui n'ont pas de données de géolocalisation.
df_rcr["LONGITUDE"].replace("null",None,inplace=True)

In [ ]:
def create_point(row):
    latitude=float(row["LATITUDE"]) if row["LATITUDE"] is not None else None
    longitude=float(row["LONGITUDE"]) if row["LONGITUDE"] is not None else None
    return Point(longitude,latitude) if latitude and longitude else None

In [ ]:
df_rcr["geometry"] =df_rcr.apply(create_point,axis=1)

In [ ]:
gdf=gpd.GeoDataFrame(df_rcr,geometry="geometry")

In [ ]:
gdf

####Création de la carte repérant les RCR qui ont des documents dans la langue souhaitée

L'objet de cette première visualitation cartographique est de repérer les RCR en question, pas de présenter des éléments quantitatifs.

####Import du fond de carte

In [ ]:
fond_de_carte=gpd.read_file("https://www.data.gouv.fr/fr/datasets/r/087ab701-f21d-4046-b53e-8b647baf505d",crs="EPSG:2154") #On charge les données de la carte de France sur data.gouv.fr, en Lambert 93.
# On part d'ici "https://www.data.gouv.fr/fr/datasets/carte-des-101-departements-francais-projetes-en-lambert-sous-la-metropole/, comme ça tous les RCR du Sudoc seront représentés.

####Mise à la norme de projection Lambert93 du *geodataframe*

In [ ]:
print(gdf.crs)#Le CRS n'est pas défini pour le geodataframe !

In [ ]:
gdf.set_crs(epsg=4326,inplace=True)#On définit bien la colonne "geometry" avec le CRS classique "longitude/latitude".

In [ ]:
print(gdf.crs) #On vérifie que c'est bon.

In [ ]:
gdf=gdf.to_crs(epsg=2154) #On convertit les coordonnées classiques selon la projection Lambert93.

In [ ]:
gdf=gdf.to_crs(fond_de_carte.crs) #On convertit la projection cartographique.

In [ ]:
gdf

####Tracé de la carte

In [ ]:
fig,ax=plt.subplots(figsize=(10,10)) #On créé la figure et les axes.
x_min,y_min,x_max,y_max=fond_de_carte.total_bounds #On limite notre étendue des axes aux limites du fond de carte.
marge=50000
ax.set_xlim(x_min-marge,x_max+marge)
ax.set_ylim(y_min-marge,y_max+marge)
fond_de_carte.plot(ax=ax,color="white",edgecolor="black") #On trace le fond de carte.
gdf.plot(ax=ax, color="blue", markersize=10) #On trace les points.
ax.set_title("Les fonds en "+langue_fr+" dans le Sudoc")

###Carte avec pondérations tenant compte du nombre de notices

Visualisation cartographique avec des données quantitatives concernant le nombre de notices à l'aide d'une *bubblemap*


In [ ]:
fig,ax=plt.subplots(figsize=(10,10)) #Pareil que tout à l'heure.
x_min,y_min,x_max,y_max=fond_de_carte.total_bounds
marge = 50000
ax.set_xlim(x_min - marge, x_max + marge)
ax.set_ylim(y_min - marge, y_max + marge)
fond_de_carte.plot(ax=ax, color="white", edgecolor="black")

gdf.plot(ax=ax,column="Notices",cmap="YlGnBu",scheme="natural_breaks",legend=True,markersize=gdf["Notices"],alpha=0.7)  #On trace les points avec la taille basée sur la colonne "Notices". On utilise une échelle de couleurs du jaune vers le bleu.
ax.set_title("Bubble map des fonds en "+langue_fr+" dans le Sudoc")

###Diagramme en barres des 25 premières bibliothèques posssédant des fonds dans la langue choisie en termes de nombre de notices

In [ ]:
df_top_25=df_rcr.sort_values("Notices",ascending=False).head(25) #On veut le top 25 du Sudoc en termes de nombres de notices.
plt.figure(figsize=(12,6))
plt.bar(df_top_25["RCR"], df_top_25["Notices"])
plt.xlabel("RCR")
plt.ylabel("Nombre de notices")
plt.title("Top 25 des RCR par nombre de notices de documents en "+langue_fr)
plt.xticks(rotation=45)
plt.tight_layout()